É uma sequência de camadas e podemos usar o modelo sequencial oferecido pelo Keras, que possui as funções necessárias para construir cada camada de uma rede neural convolucional

O Keras utiliza o TensorFlow como backend, pois, na prática, O Keras é apenas uma biblioteca para simplificar a complexidade do TensorFlow. Aqui estão as versões utilizadas:

In [ ]:
#Imports
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
import os

Datasets paths

In [ ]:
train_dir = '../dataset/train/'
test_dir = '../dataset/test1/'
print(os.listdir("../dataset"))

Image properties

In [ ]:
FAST_RUN = False
# Largura imagem
Image_Width = 128
# Altura imagem
Image_Height = 128
# Tamanho da imagem
Image_Size = (Image_Width, Image_Height)
# Canais de imagem
Image_Channels = 3

Prepare dataset for training model:

In [ ]:
filenames = os.listdir(train_dir)
categories = []
for file in filenames:
    category = file.split('.')[0]    
    if category == 'dog':
        categories.append(1)
    else:     
        categories.append(0)

df = pd.DataFrame({
    'filename' : filenames,
    'category' : categories
    })

In [ ]:
df.head()

In [ ]:
df.tail()

Total de arquivos

In [ ]:
df['category'].value_counts().plot.bar()

Amostra de imagem

In [ ]:
sample = random.choice(filenames)
image = load_img(train_dir + sample)
plt.imshow(image)

Inicializando a rede neural convolucional

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization, Dense
from tensorflow.keras.layers import Activation

model = Sequential()

Defininco parâmetros para o shape dos dados de entrada e a função de ativação. Usaremos 32 features para um array 2D e definiremos nosso array como o formato 3x3

Converteremos todas as nossas imagens 64x64 pixels em um array 3D (pois as imagens são coloridas com 3 canais de cores)

In [ ]:
## Conv_1
model.add(Conv2D(32, (3, 3), activation = 'relu', input_shape=
                (128, 128, 3))
          )
model.add(keras.layers.Conv2D(32,
                              (3,3),
                              input_shape=(Image_Width, Image_Height, Image_Channels),
                              activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

In [ ]:
## Conv_2
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))

In [ ]:
## Conv_3
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))

In [ ]:
## Flatten
model.add(Flatten())
model.add(Dense(512, activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(2, activation = 'softmax'))

Em seguida, aplicamos o agrupamento (pooling) para reduzir o tamanho do mapa de features, resultado da primeria camada de convolução (dividido por 2):

Adicionamos, então, a segunda camada de convolução, tornando nossa rede um pouco mais profunda:

Mais uma vez, aplicamos a camada de pooling à saída da camada de convolução anterior

Agora, aplicamos o "achatamento", ou apenas Flatten, para converter a estrutura de dados 2D, resultando da camada anterior em uma estrutura 1D, ou seja, um vetor

No próximo passo, conectamos todas as camadas. Usamos uma função de ativação retificadora (relu) e então uma função de ativação sigmoide, para obter as probabilidados de cada imagem conter um cachorro ou um gato. O modelo raramente terá 100% de certeza e o que ele gera como resultado é uma probabilidade.

In [ ]:
## Optimizer and loss
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'rmsprop',              
              metrics = ['accuracy']
              )

In [ ]:
## Analyzing model
model.summary()

Finalmente, compilamos nossa rede neural. Para compilar a rede, usamos o otimizador "Adam", um excelente algoritmo para otimização baseada em gradiente de funções objetivas, estocásticas, que toma como base uma estimativa adaptada de momneto de baixa ordem.

Usamos uma função log loss com "entropia binária cruzada", pois ela funciona bem com funções sigmoides. Nossa métrica será a acurácia, pois essa é nossa maior preocupação no treinamento desse tipo de modelo.

Nesse ponto, temos nossa rede contruída. Precisamos, agora, treiná-la

### Treinando a rede neural convolucional

### Pré-processamento

Para essa tarefa, vamos usar a função "ImageDataGenerator()" do Keras e ajustar escala e zoom das imagens de treino e a escala das imagens de validação.

In [ ]:
## Define callbacks and learning rate:
from tensorflow.keras.callbacks import EarlyStopping    
from tensorflow.keras.callbacks import ReduceLROnPlateau

earlystop = EarlyStopping(patience = 10)
learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',
                                            patience = 2,
                                            verbose = 1,
                                            factor = 0.5,
                                            min_lr = 0.00001
                                            )
callbacks = [earlystop, learning_rate_reduction]

Aplicamos, então, os dois objetos criados anteriormente para pré-processar os dados de treino e validação. Lembre-se: o tratamento aplicado aos dados de validação deve ser o mesmo aplicado aos dados de treino.

In [ ]:
## Manage data:
df["category"] = df["category"].replace({ 0 : 'cat', 1 : 'dog' })    
train_df, validate_df = train_test_split(df, test_size = 0.30,
                                         random_state = 42)

train_df = train_df.reset_index(drop = True)
validate_df = validate_df.reset_index(drop = True)

In [ ]:
train_df['category'].value_counts().plot.bar()

In [ ]:
validate_df['category'].value_counts().plot.bar()

In [ ]:
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]
batch_size = 15

### Treinamento

Usaremos 8.000 passos em nosso conjunto de treinamento para cada época. Escolhemos 2.000 etapas de validação para as imagens de validação - esses hiperparâmetros são definidos por você

In [ ]:
## Training and validation data generator:
train_datagen = ImageDataGenerator(rotation_range = 15,
                                   rescale = 1./255,
                                   shear_range = 0.1,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   )
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    train_dir,
                                                    x_col = 'filename',
                                                    y_col = 'category',
                                                    target_size = Image_Size,
                                                    class_mode = 'categorical',
                                                    batch_size = batch_size
                                                    )

In [ ]:
validation_datagen  = ImageDataGenerator(rescale = 1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df,
    train_dir,
    x_col = 'filename',
    y_col = 'category',
    target_size = Image_Size,
    class_mode = 'categorical',
    batch_size = batch_size
    )

See how our generator work

In [ ]:
example_df = train_df.sample(n = 1).reset_index(drop = True)
example_generator = train_datagen.flow_from_dataframe(
    example_df,
    train_dir,
    x_col = 'filename',
    y_col = 'category',
    target_size = Image_Size,
    class_mode = 'categorical')

In [ ]:
plt.figure(figsize = (12, 12))
for i in range(0, 15):
    plt.subplot(5, 3, i+1)
    for X_batch, Y_batch in example_generator:
        image = X_batch[0]
        plt.imshow(image)
        break
plt.tight_layout()        
plt.show

In [ ]:
epochs = 3 if FAST_RUN else 50
history = model.fit(
    train_generator,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = total_validate//batch_size,
    steps_per_epoch = total_validate//batch_size,
    callbacks = callbacks    
    )        

In [ ]:
## Test data preparation:
test_filenames = os.listdir(test_dir)
test_df = pd.DataFrame({
    'filename' : test_filenames
    })
nb_samples = test_df.shape[0]

test_datagen = ImageDataGenerator(rotation_range = 15,
                                  rescale = 1./255,                                  
                                  shear_range = 0.1,
                                  zoom_range = 0.2,
                                  horizontal_flip = True,
                                  width_shift_range = 0.1,
                                  height_shift_range = 0.1
                                  )
test_generator = test_datagen.flow_from_dataframe(test_df,
                                                  test_dir,
                                                  x_col = 'filename',
                                                  y_col = None,
                                                  target_size = Image_Size,
                                                  class_mode = None,
                                                  batch_size = batch_size
                                                  )

In [ ]:
## Save the model:
model.save("model_cats_dogs.h5")

Virtualize Training

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (12, 12))
ax1.plot(history.history['loss'], 
         color = 'b', 
         label = "Training loss")
ax1.plot(history.history['val_loss'], 
         color = 'r',
         label = "validation loss")
ax1.set_xticks(np.arange(1, epochs, 1))
ax1.set_yticks(np.arange(0, 1, 0.1))

ax2.plot(history.history['acc'],
         color = 'b',
         label = "Training accuracy")
ax2.plot(history.history['val_acc'],
         color = 'r',
         label = "Validation accuracy")
ax2.xticks(np.arange(1, epochs, 1))

legend = plt.legend(loc = 'best', shadow = Tue)
plt.tight_layout()
plt.show()

In [ ]:
## Make categorical prediction
predict = model.predict(test_generator,
                        steps = np.ceil(nb_samples/batch_size))

In [ ]:
## Convert labels to categories
test_df['category'] = np.argmax(predict, axis = -1)

label_map = dict((v, k) for k, v in train_generator.class_indices.items())
test_df['category'] = test_df['category'].replace(label_map)

test_df['category'] = test_df['category'].replace({'dog' : 1, 'cat' : 0})

Virtualize result

In [ ]:
test_df['category'].value_counts().plot.bar()

In [ ]:
## Visualize the prediction results
sample_test = test_df.head(18)
sample_test.head()
plt.figure(figsize = (12, 24))
for index, row in sample_test.iterrows():
    filename = row['filename']
    category = row['category']
    img = load_img(test_dir + filename,
                   target_size = Image_Size)
    plt.subplot(6, 3, index + 1)
    plt.imshow(img)
    plt.xlabel(filename + '(' + "{}".format(category) + ')')

plt.tight_layout()    
plt.show()

Submission

In [ ]:
submission_df = test_df.copy()
submission_df['id'] = submission_df['filename'].str.split('.').str[0]
submission_df['label'] = submission_df['category']
submission_df.drop(['filename', 'category'], 
                   axis = 1, 
                   inplace = True)
submission_df.to_csv('submission.csv', index = False)